In [ ]:
import sys
import logging
import re
import psycopg2
import requests
import pandas as pd
from fredapi import Fred
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database

from docopt import docopt

from pprint import pprint as pp


__version__ = '0.0.1'


def get_db_engine(db_name, user, password, host, port=5432):

    db_connection = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        database=db_name,
        user=user,
        password=password,
        host=host,
        port=port
    )

    engine = create_engine(db_connection)
    if not database_exists(engine.url):
        try:
            create_database(engine.url)

        except Exception as exc:
            logger.exception('Could not create database. \nException: %r', exc)
            return exc
    return engine


def main():

    user = 'donal'
    password = 'donal'
    host = 'localhost'
    api_key = '01af77900eb060649a7c504ee0705b4d'
    db_name = 'fred_db'
    port = 5432

    engine = get_db_engine(db_name, user, password, host, port)

    fred = Fred(api_key=api_key)
    gdp = fred.get_series('GDPC1').rename('gdp')  # ('GDP')
    um_cust_sent_index = fred.get_series('UMCSENT').rename('umcsent')  # ('UM customer sentiment index')
    us_civ_unemploy_rate = fred.get_series('UNRATE').rename('unrate')  # ('US  Civilian Unemployment Rate')
    
    frames = [gdp, um_cust_sent_index, us_civ_unemploy_rate]
    fred_data = pd.concat(frames, axis=1)
    
    for row in fred_data.iterrows():
        print(row)
    return fred_data

if __name__ == "__main__":
    data = main()
    print(data)

In [ ]:
for index, row in data.iterrows():
        print(index, row[0], row[1], row[2])
        break
    

In [ ]:
import sys
import logging
import re
import psycopg2
import requests
import pandas as pd
from fredapi import Fred
from sqlalchemy import create_engine, text
from sqlalchemy_utils import database_exists, create_database

from docopt import docopt

from pprint import pprint as pp


__version__ = '0.0.1'


def get_db_engine(db_name, user, password, host, port=5432):

    db_connection = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        database=db_name,
        user=user,
        password=password,
        host=host,
        port=port
    )

    engine = create_engine(db_connection)
    if not database_exists(engine.url):
        try:
            create_database(engine.url)

        except Exception as exc:
            logger.exception('Could not create database. \nException: %r', exc)
            return exc
    return engine


def main():

    user = 'donal'
    password = 'donal'
    host = 'localhost'
    db_name = 'fred_db'
    table_name = 'new_table2'
    port = 5432
    load_preference = 'append'  # 'replace'
    
    engine = get_db_engine(db_name, user, password, host, port)
    
    print('check 1')
    '''
    create_command = text(
        'CREATE TABLE table_name (date date, gdp decimal, umcsent decimal, unrate decimal)'
    )
    engine.execute(create_command, table_name=table_name)
    '''
    
    
    fred_data = pd.DataFrame({'S1':{'Q1':0.90,'Q2':90,'Q3':0.90,'Q4':0.90},
                              'S2':{'Q1':0.90,'Q2':90,'Q3':0.10,'Q4':0.10},
                              'S3':{'Q1':0.10,'Q2':10,'Q3':0.10,'Q4':0.10},
                              'S4':{'Q1':0.10,'Q2':10,'Q3':0.10,'Q4':0.10}})
    print('fred_data', fred_data)
    fred_data = fred_data.to_dict('records')
    
    print('fred_data', type(fred_data))
    print('fred_data', fred_data)
    for item in fred_data:
        print('type(item)', type(item))
        print('item', item)
        break
    
    # Insert the data into an SQL table
    # fixme: add option to replace or append here depending on initial or incremental load type - update replace
    #fred_data.to_sql(name=table_name, con=engine, if_exists=load_preference, index=True, index_label='timestamp')

if __name__ == "__main__":
    main()

In [ ]:
# -*- coding: utf-8 -*-

"""
Retrieve statistics from the FRED api

Usage:
    fred_stats.py --user=USER --password=PASSWORD ( --initial | --incremental ) [--start=START]
                  [--end=END] [--database=DATABASE] [--table=TABLE]
                  [--host=HOST] [--api_key=API_KEY] [--port=PORT]
    fred_stats.py (-h | --help)
    fred_stats.py (-v | --version)

Options:
    -u --user=USER              Your Postgres username.
    -p --password=PASSWORD      Your Postgres password.
    --initial                   Use initial loading.
    --incremental               Use incremental loading.
    -h --help                   Show this screen.
    -v --version                Show version.

Additional options:
    -s --start=START            yyyy. Start year for average unemployment rate aggregation.
                                Default = 1980
    -e --end=END                yyyy. End year for average unemployment rate aggregation.
                                Default = 2015
    -d --database=DATABASE      The name of the database in which you wish to store the Fred data.
                                If the database does not exist it will be created.
                                Default = 'fred_db'.
    -t --table=TABLE            The name of the table in which you wish to insert/update the data.
                                Default = 'fred_data
    -a --api_key=API_KEY        Your FRED api key.
                                Default = Fred API key of Dónal Flanagan.
    --host=HOST                 Database hostname.
                                Default = 'localhost'.
    --port=PORT                 The Postgres port.
                                Default = 5432
"""

import sys
import logging
import pandas as pd
from fredapi import Fred
from sqlalchemy import create_engine, text, Table, Column, DateTime, Numeric, MetaData
from sqlalchemy_utils import database_exists, create_database
from docopt import docopt
from pprint import pprint as pp


logger = logging.getLogger('fred_stats')
__version__ = '0.0.1'


def get_db_engine(db_name, user, password, host, port=5432):

    db_connection = "postgresql://{user}:{password}@{host}:{port}/{database}".format(
        database=db_name,
        user=user,
        password=password,
        host=host,
        port=port
    )

    engine = create_engine(db_connection)
    if not database_exists(engine.url):
        try:
            logger.info('Database created: %s', db_name)
            create_database(engine.url)

        except Exception as exc:
            logger.exception('Could not create database. \nException: %r', exc)
            return exc
    return engine


def main():
    user = 'donal'
    password = 'donal'
    host = 'localhost'
    api_key = '01af77900eb060649a7c504ee0705b4d'
    db_name = 'fred_db'
    table_name = 'some_table'
    port = 5432
    start_date = '2000-01-01'
    end_date = '2005-12-31'

    engine = get_db_engine(db_name, user, password, host, port)

    fred_data = pd.DataFrame({'S1':{'Q1':0.90,'Q2':90,'Q3':0.90,'Q4':0.90},
                              'S2':{'Q1':0.90,'Q2':90,'Q3':0.10,'Q4':0.10},
                              'S3':{'Q1':0.10,'Q2':10,'Q3':0.10,'Q4':0.10},
                              'S4':{'Q1':0.10,'Q2':10,'Q3':0.10,'Q4':0.10}})


    metadata = MetaData(engine)
    fred_table = Table(table_name,
                       metadata,
                       Column('timestamp', DateTime),
                       Column('gdp', Numeric),
                       Column('umcsent', Numeric),
                       Column('unrate', Numeric))
    if not engine.dialect.has_table(engine, table_name):
        metadata.create_all()

    print(fred_table.name)

    print('\n----------------------------------')
    print(MetaData)



    # Query the SQL table for the average unemployment rate.
    unemployment_query = text(
        """SELECT Extract(YEAR from timestamp)::INT as year, avg(unrate)
           FROM table_name
           WHERE timestamp >= :start_date and timestamp <= :end_date
           GROUP BY year
           ORDER BY year"""
    )
    result = engine.execute(unemployment_query, start_date=start_date, end_date=end_date, table_name=fred_table.c.name)

    df = pd.DataFrame(result.fetchall())
    df.columns = result.keys()
    df.set_index('year', inplace=True)

    print('The average rate of unemployment in the USA for each year between %s and %s is as follows:' %
          (start_date, end_date))
    print(df)

if __name__ == '__main__':
    sys.exit(main())
